In [65]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from time import sleep
import pandas as pd

In [9]:
search_results = requests.get("https://datadryad.org/search?f%5Bdryad_dataset_file_ext_sm%5D%5B%5D=trees&per_page=200")
soup = BeautifulSoup(search_results.text, "html.parser")

In [12]:
h3_tags = soup.find_all("h3")

dataset_links = []

for h3 in h3_tags:
    a_tags = h3.find_all("a")
    for a in a_tags:
        href = a.get("href")
        if href:
            dataset_links.append(href)

len(dataset_links)

100

In [86]:
df_dict = {
    "dyriad_link": [],
    "dataset_name": [],
    "trees_link": [],
    "trees_name": [],
    "trees_size": [],
    "doi": [],
}
dataset_link = dataset_links[0]

for dataset_link in tqdm(dataset_links):
    response = requests.get(f"https://datadryad.org{dataset_link}")
    soup = BeautifulSoup(response.text, "html.parser")
    
    title = soup.find("h1").text.strip()

    all_links = soup.find_all("a")
    tree_links = [link for link in all_links if ".trees" in (link.get("title") or "")]

    doi = [span for span in soup.find_all("span") if "o-metadata__group2-item" in (span.get("class") or "") and "doi.org" in (span.text or "")][0].text.strip()

    for tree_link in tree_links:
        df_dict["dyriad_link"].append(dataset_link)
        df_dict["dataset_name"].append(title)
        df_dict["trees_link"].append(tree_link.get("href"))
        df_dict["trees_name"].append(tree_link.get("title"))
        try:
            df_dict["trees_size"].append(
                tree_link.find_parent("span").find_next_sibling("div").find("input").get("data-size")
            )
        except:
            df_dict["trees_size"].append(None)
        df_dict["doi"].append(doi)

    sleep(1)        

  0%|          | 0/100 [00:00<?, ?it/s]

In [85]:
df = pd.DataFrame(df_dict)
df.to_csv("data/scraped_dryad.csv", index=False, sep=";")